In [2]:
import pandas as pd

ratings = pd.read_csv('IS 477 Project/ratings.csv')
movies = pd.read_csv('IS 477 Project/movies.csv')
links = pd.read_csv('IS 477 Project/links.csv')
tags = pd.read_csv('IS 477 Project/tags.csv')
credits = pd.read_csv('IS 477 Project/credits.csv')

ratings.head(), movies.head()


(   userId  movieId  rating  timestamp
 0       1        1     4.0  964982703
 1       1        3     4.0  964981247
 2       1        6     4.0  964982224
 3       1       47     5.0  964983815
 4       1       50     5.0  964982931,
    movieId                               title  \
 0        1                    Toy Story (1995)   
 1        2                      Jumanji (1995)   
 2        3             Grumpier Old Men (1995)   
 3        4            Waiting to Exhale (1995)   
 4        5  Father of the Bride Part II (1995)   
 
                                         genres  
 0  Adventure|Animation|Children|Comedy|Fantasy  
 1                   Adventure|Children|Fantasy  
 2                               Comedy|Romance  
 3                         Comedy|Drama|Romance  
 4                                       Comedy  )

In [3]:
ratings.head(), movies.head()


(   userId  movieId  rating  timestamp
 0       1        1     4.0  964982703
 1       1        3     4.0  964981247
 2       1        6     4.0  964982224
 3       1       47     5.0  964983815
 4       1       50     5.0  964982931,
    movieId                               title  \
 0        1                    Toy Story (1995)   
 1        2                      Jumanji (1995)   
 2        3             Grumpier Old Men (1995)   
 3        4            Waiting to Exhale (1995)   
 4        5  Father of the Bride Part II (1995)   
 
                                         genres  
 0  Adventure|Animation|Children|Comedy|Fantasy  
 1                   Adventure|Children|Fantasy  
 2                               Comedy|Romance  
 3                         Comedy|Drama|Romance  
 4                                       Comedy  )

This cell gives us a first look at the structure of the two main tables: ratings and movies. It helps confirm that the columns we expect (like userId, movieId, rating, title, genres) are present and that the file paths were loaded correctly.

In [4]:
print("Ratings shape:", ratings.shape)
print("Movies shape:", movies.shape)

print("\nRatings dtypes:\n", ratings.dtypes)
print("\nMovies dtypes:\n", movies.dtypes)


Ratings shape: (100836, 4)
Movies shape: (9742, 3)

Ratings dtypes:
 userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

Movies dtypes:
 movieId     int64
title      object
genres     object
dtype: object


Here we check how many rows and columns each table has and what data type each column currently uses. This tells us whether IDs and numeric fields are already in a good format, and it informs what “standardizing data types” we still need to do.

In [5]:
ratings['userId'] = ratings['userId'].astype('int64')
ratings['movieId'] = ratings['movieId'].astype('int64')
ratings['rating'] = ratings['rating'].astype('float64')
ratings['timestamp'] = ratings['timestamp'].astype('int64')

movies['movieId'] = movies['movieId'].astype('int64')
movies['title'] = movies['title'].astype('string')
movies['genres'] = movies['genres'].astype('string')


This cell explicitly sets the types for our most important columns: user IDs, movie IDs, ratings, timestamps, titles, and genres. Doing this now avoids subtle bugs later when we try to merge tables or compute statistics and makes the cleaning process reproducible.

In [6]:
ratings['rating_datetime'] = pd.to_datetime(ratings['timestamp'], unit='s')
ratings[['timestamp', 'rating_datetime']].head()


,timestamp,rating_datetime
0,964982703,2000-07-30 18:45:03
1,964981247,2000-07-30 18:20:47
2,964982224,2000-07-30 18:37:04
3,964983815,2000-07-30 19:03:35
4,964982931,2000-07-30 18:48:51


The MovieLens timestamps are stored as UNIX seconds, which are hard to interpret. We convert them into a standard datetime column so that, if we later want to analyze ratings over time, the dates are already in a clean, human-readable format.

In [7]:
print("Missing values in ratings:\n", ratings.isna().sum())
print("\nMissing values in movies:\n", movies.isna().sum())
print("\nMissing values in links:\n", links.isna().sum())
print("\nMissing values in tags:\n", tags.isna().sum())


Missing values in ratings:
 userId             0
movieId            0
rating             0
timestamp          0
rating_datetime    0
dtype: int64

Missing values in movies:
 movieId    0
title      0
genres     0
dtype: int64

Missing values in links:
 movieId    0
imdbId     0
tmdbId     8
dtype: int64

Missing values in tags:
 userId       0
movieId      0
tag          0
timestamp    0
dtype: int64


This cell counts how many missing (NaN) values appear in every column across our main tables. Knowing where missingness occurs is essential for planning how to handle it (dropping rows vs. filling values), and you can directly mention these counts in your progress report.

In [8]:
ratings_clean = ratings.dropna(subset=['userId', 'movieId', 'rating'])

movies_clean = movies.dropna(subset=['movieId', 'title'])

print("Original ratings rows:", len(ratings))
print("Cleaned ratings rows:", len(ratings_clean))
print("Original movies rows:", len(movies))
print("Cleaned movies rows:", len(movies_clean))


Original ratings rows: 100836
Cleaned ratings rows: 100836
Original movies rows: 9742
Cleaned movies rows: 9742


Here we create cleaned copies (ratings_clean, movies_clean) by removing rows that are missing critical information like IDs or titles, which would break analyses or joins. We also print how many rows were lost so you can report this as part of your “data cleaning and quality assessment.”

In [9]:
import re

def extract_year(title):
    """Extract a 4-digit year from a movie title like 'Toy Story (1995)'."""
    match = re.search(r'\((\d{4})\)', str(title))
    return int(match.group(1)) if match else None

movies_clean['release_year'] = movies_clean['title'].apply(extract_year)

movies_clean[['title', 'release_year']].head()


,title,release_year
0,Toy Story (1995),1995.0
1,Jumanji (1995),1995.0
2,Grumpier Old Men (1995),1995.0
3,Waiting to Exhale (1995),1995.0
4,Father of the Bride Part II (1995),1995.0


Many MovieLens titles encode the release year in parentheses at the end of the title; this function pulls that 4-digit year into its own release_year column. Having the year as a separate numeric column is crucial for your research question about how ratings vary by release year.

In [10]:
movies_clean = movies_clean.dropna(subset=['release_year'])
movies_clean = movies_clean[movies_clean['release_year'] >= 1900]

movies_clean['release_year'].describe()


count    9729.000000
mean     1994.613629
std        18.535219
min      1902.000000
25%      1988.000000
50%      1999.000000
75%      2008.000000
max      2018.000000
Name: release_year, dtype: float64

Some titles may not have a year, or the extracted year might be unrealistic (e.g., parsing noise). This cell removes movies with missing or nonsensical years so that later visualizations (like ratings over time) are based on reasonable values.

In [11]:
movies_clean['genres_list'] = movies_clean['genres'].apply(
    lambda x: str(x).split('|') if pd.notna(x) and x != '(no genres listed)' else []
)

movies_clean[['title', 'genres', 'genres_list']].head()


,title,genres,genres_list
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,Jumanji (1995),Adventure|Children|Fantasy,"[Adventure, Children, Fantasy]"
2,Grumpier Old Men (1995),Comedy|Romance,"[Comedy, Romance]"
3,Waiting to Exhale (1995),Comedy|Drama|Romance,"[Comedy, Drama, Romance]"
4,Father of the Bride Part II (1995),Comedy,[Comedy]


MovieLens stores multiple genres as a single string separated by |. We convert that into a genres_list column containing Python lists, which makes it much easier to later group by genre or explode the data to analyze ratings across genres.

In [12]:
print("Ratings_clean shape:", ratings_clean.shape)
print("Movies_clean shape:", movies_clean.shape)

ratings_clean.head(), movies_clean.head()


Ratings_clean shape: (100836, 5)
Movies_clean shape: (9729, 5)


(   userId  movieId  rating  timestamp     rating_datetime
 0       1        1     4.0  964982703 2000-07-30 18:45:03
 1       1        3     4.0  964981247 2000-07-30 18:20:47
 2       1        6     4.0  964982224 2000-07-30 18:37:04
 3       1       47     5.0  964983815 2000-07-30 19:03:35
 4       1       50     5.0  964982931 2000-07-30 18:48:51,
    movieId                               title  \
 0        1                    Toy Story (1995)   
 1        2                      Jumanji (1995)   
 2        3             Grumpier Old Men (1995)   
 3        4            Waiting to Exhale (1995)   
 4        5  Father of the Bride Part II (1995)   
 
                                         genres  release_year  \
 0  Adventure|Animation|Children|Comedy|Fantasy        1995.0   
 1                   Adventure|Children|Fantasy        1995.0   
 2                               Comedy|Romance        1995.0   
 3                         Comedy|Drama|Romance        1995.0   
 4          

This is a final sanity check to see the size of the cleaned tables and preview the first few rows. At this point, you’ve completed the Week 3 goals: you standardized data types, converted timestamps to dates, handled missing values in key columns, and derived structured fields like release_year and genres_list.

In [13]:
ratings_clean.to_csv('IS 477 Project/ratings_clean.csv', index=False)
movies_clean.to_csv('IS 477 Project/movies_clean.csv', index=False)
